In [1]:
%display latex

In [2]:
N = 5
F = GF(2^N, 'x')
x = F.gen()

In [3]:
F.polynomial()

x^5 + x^2 + 1

In [4]:
# How did I find this basis? Jungnickel's paper and brute force...
basis = [x^3, x^5, x^11, x^22, x^24]

In [5]:
def checkSelfDual(basis):
    for a in basis:
        for b in basis:
            if a == b:
                if (a * b).trace() != F(1):
                    raise Exception(a,b)
            else:
                if (a * b).trace() != F(0):
                    raise Exception(a,b)

In [6]:
checkSelfDual(basis)

In [7]:
# express an element as a linear combination of the basis
components = lambda k: [(k * el).trace() for el in basis]

# additive group character
chi = lambda k: exp(pi * I * int(k.trace()))

def toInt(k):
    return list(F).index(k)

In [8]:
y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

In [9]:
# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

In [10]:
# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

In [11]:
def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

In [12]:
for mu in F:
    curve = lambda t: mu * t # \beta = \mu \alpha
    sols = sign_perm(solve_basis(basis, curve), [1,1,1,1,1])
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
print('Recurrence relation holds!')

Recurrence relation holds!


In [13]:
def ray(mu):
    return lambda t: mu * t

In [14]:
def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [104]:
ps = PS(ray, [[1,1,1,1,1]] * 32)
ps

32 x 32 dense matrix over Symbolic Ring (use the '.str()' method to see the entries)

Kantor's spread which is not isomorphic to the Desarguesian spread is given by the opperation:
$$
\alpha = 0,
\quad
\beta = m^2 \alpha + m T(\alpha) + T(m\alpha),
$$
for $m \in GF(2^5)$ and $T$ is the field trace function.

In [16]:
def Kantor(m):
    return lambda a: m^2 * a + m * a.trace() + (m * a).trace()

Let's verify that this operation produces an affine plane, i.e., each line is a subspace and there is no non-trivial intersection between lines.

In [17]:
def checkAdditivity(points):
    for a1, b1 in points:
        for a2, b2 in points:
            if (a1 + a2, b1 + b2) not in points:
                raise Exception('Not additive!')

In [18]:
for m in F:
    points = [(a, Kantor(m)(a)) for a in F]
    checkAdditivity(points)

In [19]:
for m1 in F:
    for m2 in F:
        if m1 != m2:
            curve1 = [(a, Kantor(m1)(a)) for a in F]
            curve2 = [(a, Kantor(m2)(a)) for a in F]
            if len(list(set(curve1) & set(curve2))) != 1:
                raise Exception('Found non-trivial intersection!')

So it is defined perfectly. Now let's test to see if any of the curves are abelian given our choice of phase space $\Gamma$.

In [20]:
def Proj(u, v=None):
    if not v:
        v = u
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(2^N)

def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

In [21]:
def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

def D(a, b):
    return phi(a, b) * Z(a) * X(b)

In [22]:
def testCurve(points):
    for p1 in points:
        for p2 in points:
            p_op = D(p1[0], p1[1]) @ D(p2[0], p2[1])
            s_op = D(p1[0] + p2[0], p1[1] + p2[1]) # assuming additivity
            if not np.allclose(p_op, s_op):
                raise Exception(p1, p2)
    return True

Takes way too long, time to bring `numpy`.

In [23]:
import numpy as np

In [24]:
chi_np = lambda k: np.exp(np.pi * 1j * int(k.trace()))

def Proj(u, v=None):
    if not v:
        v = u
    return np.outer(u, v.conj().T)

Id = np.eye(2^N)

def Fourier():
    s = np.zeros((2^N, 2^N), dtype='complex128')
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi_np(a * b) / np.sqrt(2^N)
    return s
FF = Fourier()

def phi(a, b):
    return complex(ps[toInt(a), toInt(b)])

def Z(a):
    return np.diag([chi_np(a * k) for k in F])

def X(b):
    return FF.conj().T @ Z(b) @ FF

def D(a, b):
    return phi(a, b) * Z(a) @ X(b)

In [24]:
points = [(a, Kantor(F(0))(a)) for a in F]
testCurve(points)

True

In [25]:
for m in F:
    print('Testing with m = {}'.format(m))
    points = [(a, Kantor(m)(a)) for a in F]
    try:
        testCurve(points)
        print(m, 'abelian')
    except:
        continue

Testing with m = 0
0 abelian
Testing with m = x
Testing with m = x^2
Testing with m = x^3
Testing with m = x^4
Testing with m = x^2 + 1
Testing with m = x^3 + x
Testing with m = x^4 + x^2
Testing with m = x^3 + x^2 + 1
Testing with m = x^4 + x^3 + x
Testing with m = x^4 + 1
Testing with m = x^2 + x + 1
Testing with m = x^3 + x^2 + x
Testing with m = x^4 + x^3 + x^2
Testing with m = x^4 + x^3 + x^2 + 1
Testing with m = x^4 + x^3 + x^2 + x + 1
Testing with m = x^4 + x^3 + x + 1
Testing with m = x^4 + x + 1
Testing with m = x + 1
Testing with m = x^2 + x
Testing with m = x^3 + x^2
Testing with m = x^4 + x^3
Testing with m = x^4 + x^2 + 1
Testing with m = x^3 + x^2 + x + 1
Testing with m = x^4 + x^3 + x^2 + x
Testing with m = x^4 + x^3 + 1
Testing with m = x^4 + x^2 + x + 1
Testing with m = x^3 + x + 1
Testing with m = x^4 + x^2 + x
Testing with m = x^3 + 1
Testing with m = x^4 + x
Testing with m = 1
1 abelian


No non-trivial abelian curves found! Does this have to do with the fact that the affine planes are not inequivalent?

Let's take a non-abelian curve from the Kantor set, and switch up all the phases to see if any match.

In [39]:
for mu in F:
    curve = Kantor(mu) # \beta = \mu \alpha
    sols = sign_perm(solve_basis(basis, curve), [1,1,1,1,1])
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
print('Recurrence relation holds!')

Recurrence relation holds!


In [40]:
from itertools import product

In [43]:
mu = x
curve = Kantor(mu) 
for perm in product([1,-1], repeat=5):
    sols = sign_perm(solve_basis(basis, curve), perm)
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
    print('Recurrence relations holds for', perm)

Recurrence relations holds for (1, 1, 1, 1, 1)
Recurrence relations holds for (1, 1, 1, 1, -1)
Recurrence relations holds for (1, 1, 1, -1, 1)
Recurrence relations holds for (1, 1, 1, -1, -1)
Recurrence relations holds for (1, 1, -1, 1, 1)
Recurrence relations holds for (1, 1, -1, 1, -1)
Recurrence relations holds for (1, 1, -1, -1, 1)
Recurrence relations holds for (1, 1, -1, -1, -1)
Recurrence relations holds for (1, -1, 1, 1, 1)
Recurrence relations holds for (1, -1, 1, 1, -1)
Recurrence relations holds for (1, -1, 1, -1, 1)
Recurrence relations holds for (1, -1, 1, -1, -1)
Recurrence relations holds for (1, -1, -1, 1, 1)
Recurrence relations holds for (1, -1, -1, 1, -1)
Recurrence relations holds for (1, -1, -1, -1, 1)
Recurrence relations holds for (1, -1, -1, -1, -1)
Recurrence relations holds for (-1, 1, 1, 1, 1)
Recurrence relations holds for (-1, 1, 1, 1, -1)
Recurrence relations holds for (-1, 1, 1, -1, 1)
Recurrence relations holds for (-1, 1, 1, -1, -1)
Recurrence relations

In [45]:
mu = x
curve = Kantor(mu)
perm = [-1,-1,-1,-1,-1]
sols = sign_perm(solve_basis(basis, curve), perm)

def D2(a,b):
    return complex(c(a, curve, sols)) * (Z(a) @ X(b))

def testCurve(points):
    for p1 in points:
        for p2 in points:
            p_op = D2(p1[0], p1[1]) @ D2(p2[0], p2[1])
            s_op = D2(p1[0] + p2[0], p1[1] + p2[1]) # assuming additivity
            if not np.allclose(p_op, s_op):
                raise Exception(p1, p2)
    return True

In [46]:
testCurve([(k,curve(k)) for k in F])

True

In [53]:
ray_phases = [ps[toInt(k), toInt(curve(k))] for k in F]
for perm in product([1,-1], repeat=5):
    sols = sign_perm(solve_basis(basis, curve), perm)
    phases = [c(k, curve, sols) for k in F]
    if ray_phases == phases:
        print('Permutation produces matching phases!', perm)

Of course since the curve is not abelian in the ray DPS, it cannot be made abelian by a different choice of signs. Now let's test some different DPS sign distributions.

In [63]:
mu = x
points = [(k, Kantor(mu)(k)) for k in F]

In [64]:
for pe in product([1,-1], repeat=5):
    perm = [pe] * 32
    ps3 = PS(ray, perm)
    
    def D3(a, b):
        return complex(ps3[toInt(a), toInt(b)]) * Z(a) @ X(b)

    try:
        for p1 in points:
            for p2 in points:
                p_op = D3(p1[0], p1[1]) @ D3(p2[0], p2[1])
                s_op = D3(p1[0] + p2[0], p1[1] + p2[1])
                if not np.allclose(p_op, s_op):
                    raise Exception(p1, p2)
    except:
        print('Not abelian for permutation', pe)
        continue

    print('Found abelian curve for permutation', pe)

Not abelian for permutation (1, 1, 1, 1, 1)
Not abelian for permutation (1, 1, 1, 1, -1)
Not abelian for permutation (1, 1, 1, -1, 1)
Not abelian for permutation (1, 1, 1, -1, -1)
Not abelian for permutation (1, 1, -1, 1, 1)
Not abelian for permutation (1, 1, -1, 1, -1)
Not abelian for permutation (1, 1, -1, -1, 1)
Not abelian for permutation (1, 1, -1, -1, -1)
Not abelian for permutation (1, -1, 1, 1, 1)
Not abelian for permutation (1, -1, 1, 1, -1)
Not abelian for permutation (1, -1, 1, -1, 1)
Not abelian for permutation (1, -1, 1, -1, -1)
Not abelian for permutation (1, -1, -1, 1, 1)
Not abelian for permutation (1, -1, -1, 1, -1)
Not abelian for permutation (1, -1, -1, -1, 1)
Not abelian for permutation (1, -1, -1, -1, -1)
Not abelian for permutation (-1, 1, 1, 1, 1)
Not abelian for permutation (-1, 1, 1, 1, -1)
Not abelian for permutation (-1, 1, 1, -1, 1)
Not abelian for permutation (-1, 1, 1, -1, -1)
Not abelian for permutation (-1, 1, -1, 1, 1)
Not abelian for permutation (-1, 1

Not abelian for any repeated permutation. But what about Sainz idea. By fixing the first basis solution to be positive we obtain:

In [68]:
for mu in list(F)[1:-2]:
    print('Testing curve', mu)
    curve = Kantor(mu)
    points = [(k, curve(k)) for k in F]

    # loop through 32 different DPS
    for pe in product([1,-1], repeat=5):
        perm = [pe] * 32
        ps3 = PS(ray, perm)
        
        def D3(a, b):
            return complex(ps3[toInt(a), toInt(b)]) * Z(a) @ X(b)

        # check if the current curve is abelian in any of these
        try:
            for p1 in points:
                for p2 in points:
                    p_op = D3(p1[0], p1[1]) @ D3(p2[0], p2[1])
                    s_op = D3(p1[0] + p2[0], p1[1] + p2[1])
                    if not np.allclose(p_op, s_op):
                        raise Exception(p1, p2)
        except:
            continue
    
        print('Found abelian curve for permutation', pe, mu)

Testing curve x
Testing curve x^2
Testing curve x^3
Testing curve x^4
Testing curve x^2 + 1
Testing curve x^3 + x
Testing curve x^4 + x^2
Testing curve x^3 + x^2 + 1
Testing curve x^4 + x^3 + x
Testing curve x^4 + 1
Testing curve x^2 + x + 1
Testing curve x^3 + x^2 + x
Testing curve x^4 + x^3 + x^2
Testing curve x^4 + x^3 + x^2 + 1
Testing curve x^4 + x^3 + x^2 + x + 1
Testing curve x^4 + x^3 + x + 1
Testing curve x^4 + x + 1
Testing curve x + 1


KeyboardInterrupt: 

Exception ignored in: 'sage.symbolic.expression.py_is_exact'
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 402, in parent
  File "src/cysignals/signals.pyx", line 310, in cysignals.signals.python_check_interrupt
KeyboardInterrupt: 


KeyboardInterrupt: 

---
For Sainz

In [25]:
def powers(pts):
    return [(toInt(p[0]), toInt(p[1])) for p in pts]

Kantor's spread which is not isomorphic to the Desarguesian spread is given by the opperation:
$$
\alpha = 0,
\quad
\beta = m^2 \alpha + m T(\alpha) + T(m\alpha),
$$
for $m \in GF(2^5)$ and $T$ is the field trace function.

In [26]:
for p in powers([(k, Kantor(x)(k)) for k in F]):
    if p[0] == 0:
        a = '0'
    else:
        a = 'x'
    if p[1] == 0:
        b = '0'
    else:
        b = 'x'
    print('({}^{}, {}^{})'.format(a, p[0], b, p[1]))

(0^0, 0^0)
(x^1, x^3)
(x^2, x^10)
(x^3, x^11)
(x^4, x^27)
(x^5, x^26)
(x^6, x^23)
(x^7, x^9)
(x^8, x^4)
(x^9, x^2)
(x^10, x^8)
(x^11, x^15)
(x^12, x^24)
(x^13, x^14)
(x^14, x^16)
(x^15, x^17)
(x^16, x^1)
(x^17, x^5)
(x^18, x^12)
(x^19, x^25)
(x^20, x^28)
(x^21, x^20)
(x^22, x^13)
(x^23, x^21)
(x^24, x^22)
(x^25, x^6)
(x^26, x^7)
(x^27, x^29)
(x^28, x^30)
(x^29, x^31)
(x^30, x^18)
(x^31, x^19)


In [27]:
powers([(k, Kantor(x)(k)) for k in F])

[(0, 0),
 (1, 3),
 (2, 10),
 (3, 11),
 (4, 27),
 (5, 26),
 (6, 23),
 (7, 9),
 (8, 4),
 (9, 2),
 (10, 8),
 (11, 15),
 (12, 24),
 (13, 14),
 (14, 16),
 (15, 17),
 (16, 1),
 (17, 5),
 (18, 12),
 (19, 25),
 (20, 28),
 (21, 20),
 (22, 13),
 (23, 21),
 (24, 22),
 (25, 6),
 (26, 7),
 (27, 29),
 (28, 30),
 (29, 31),
 (30, 18),
 (31, 19)]

In [28]:
for p in [(k, Kantor(x)(k)) for k in F]:
    print(p)

(0, 0)
(x, x^3)
(x^2, x^4 + 1)
(x^3, x^2 + x + 1)
(x^4, x^3 + x + 1)
(x^2 + 1, x^4 + x^2 + x + 1)
(x^3 + x, x^3 + x^2 + x + 1)
(x^4 + x^2, x^4 + x^3 + x)
(x^3 + x^2 + 1, x^4)
(x^4 + x^3 + x, x^2)
(x^4 + 1, x^3 + x^2 + 1)
(x^2 + x + 1, x^4 + x^3 + x^2 + x + 1)
(x^3 + x^2 + x, x^4 + x^3 + x^2 + x)
(x^4 + x^3 + x^2, x^4 + x^3 + x^2 + 1)
(x^4 + x^3 + x^2 + 1, x^4 + x^3 + x + 1)
(x^4 + x^3 + x^2 + x + 1, x^4 + x + 1)
(x^4 + x^3 + x + 1, x)
(x^4 + x + 1, x^2 + 1)
(x + 1, x^3 + x^2 + x)
(x^2 + x, x^4 + x^3 + 1)
(x^3 + x^2, x^4 + x^2 + x)
(x^4 + x^3, x^3 + x^2)
(x^4 + x^2 + 1, x^4 + x^3 + x^2)
(x^3 + x^2 + x + 1, x^4 + x^3)
(x^4 + x^3 + x^2 + x, x^4 + x^2 + 1)
(x^4 + x^3 + 1, x^3 + x)
(x^4 + x^2 + x + 1, x^4 + x^2)
(x^3 + x + 1, x^3 + 1)
(x^4 + x^2 + x, x^4 + x)
(x^3 + 1, 1)
(x^4 + x, x + 1)
(1, x^2 + x)


Sum table.

In [32]:
np.chararray((2,2))

chararray([['', ''],
           ['', '']], dtype='|S1')

In [47]:
sum_table = np.chararray((2^N, 2^N), unicode=True, itemsize=5)
for a in F:
    for b in F:
        s = a + b
        if s == F(0):
            sum_table[toInt(a), toInt(b)] = '0'
        else:
            sum_table[toInt(a), toInt(b)] = 'x^{}'.format(toInt(s))
sum_table

chararray([['0', 'x^1', 'x^2', ..., 'x^29', 'x^30', 'x^31'],
           ['x^1', '0', 'x^19', ..., 'x^27', 'x^4', 'x^18'],
           ['x^2', 'x^19', '0', ..., 'x^8', 'x^28', 'x^5'],
           ...,
           ['x^29', 'x^27', 'x^8', ..., '0', 'x^16', 'x^3'],
           ['x^30', 'x^4', 'x^28', ..., 'x^16', '0', 'x^17'],
           ['x^31', 'x^18', 'x^5', ..., 'x^3', 'x^17', '0']], dtype='<U5')

In [51]:
import pandas as pd

In [52]:
DF = pd.DataFrame(sum_table)
DF.to_csv("sum_table_F_2_5.csv")

In [64]:
sum_table[20, 30]

'x^24'

In [66]:
toInt(x^20 + x^30)

24

In [68]:
toInt(x^10 + x^18)

30

In [69]:
x^30

x^4 + x

In [80]:
for k in F:
    if k == 0:
        print(0, '\t', F(0).trace())
    else:
        print('x^{}'.format(toInt(k)), '\t', k.trace())

0 	 0
x^1 	 0
x^2 	 0
x^3 	 1
x^4 	 0
x^5 	 1
x^6 	 1
x^7 	 0
x^8 	 0
x^9 	 1
x^10 	 1
x^11 	 1
x^12 	 1
x^13 	 1
x^14 	 0
x^15 	 0
x^16 	 0
x^17 	 1
x^18 	 1
x^19 	 0
x^20 	 1
x^21 	 1
x^22 	 1
x^23 	 0
x^24 	 1
x^25 	 0
x^26 	 1
x^27 	 0
x^28 	 0
x^29 	 0
x^30 	 0
x^31 	 1


Sainz wants me to change the sign for $\sigma^5$ for $\mu = \sigma^{12}, \sigma^{19}, \sigma^{23}, \sigma$.

In [122]:
perms = [[1,1,1,1,1], [1,-1,1,1,1]] + [[1,1,1,1,1]] * 10 + [[1,-1,1,1,1]] + [[1,1,1,1,1]] * 6 + [[1,-1,1,1,1]] + [[1,1,1,1,1]] * 3 + [[1,-1,1,1,1]] + [[1,1,1,1,1]] * 8
for k, pe in enumerate(perms):
    print(k, pe)

0 [1, 1, 1, 1, 1]
1 [1, -1, 1, 1, 1]
2 [1, 1, 1, 1, 1]
3 [1, 1, 1, 1, 1]
4 [1, 1, 1, 1, 1]
5 [1, 1, 1, 1, 1]
6 [1, 1, 1, 1, 1]
7 [1, 1, 1, 1, 1]
8 [1, 1, 1, 1, 1]
9 [1, 1, 1, 1, 1]
10 [1, 1, 1, 1, 1]
11 [1, 1, 1, 1, 1]
12 [1, -1, 1, 1, 1]
13 [1, 1, 1, 1, 1]
14 [1, 1, 1, 1, 1]
15 [1, 1, 1, 1, 1]
16 [1, 1, 1, 1, 1]
17 [1, 1, 1, 1, 1]
18 [1, 1, 1, 1, 1]
19 [1, -1, 1, 1, 1]
20 [1, 1, 1, 1, 1]
21 [1, 1, 1, 1, 1]
22 [1, 1, 1, 1, 1]
23 [1, -1, 1, 1, 1]
24 [1, 1, 1, 1, 1]
25 [1, 1, 1, 1, 1]
26 [1, 1, 1, 1, 1]
27 [1, 1, 1, 1, 1]
28 [1, 1, 1, 1, 1]
29 [1, 1, 1, 1, 1]
30 [1, 1, 1, 1, 1]
31 [1, 1, 1, 1, 1]


In [123]:
ps2 = PS(ray, perms)
ps2

32 x 32 dense matrix over Symbolic Ring (use the '.str()' method to see the entries)

In [124]:
points = [(k, Kantor(x)(k)) for k in F]
points

[(0, 0),
 (x, x^3),
 (x^2, x^4 + 1),
 (x^3, x^2 + x + 1),
 (x^4, x^3 + x + 1),
 (x^2 + 1, x^4 + x^2 + x + 1),
 (x^3 + x, x^3 + x^2 + x + 1),
 (x^4 + x^2, x^4 + x^3 + x),
 (x^3 + x^2 + 1, x^4),
 (x^4 + x^3 + x, x^2),
 (x^4 + 1, x^3 + x^2 + 1),
 (x^2 + x + 1, x^4 + x^3 + x^2 + x + 1),
 (x^3 + x^2 + x, x^4 + x^3 + x^2 + x),
 (x^4 + x^3 + x^2, x^4 + x^3 + x^2 + 1),
 (x^4 + x^3 + x^2 + 1, x^4 + x^3 + x + 1),
 (x^4 + x^3 + x^2 + x + 1, x^4 + x + 1),
 (x^4 + x^3 + x + 1, x),
 (x^4 + x + 1, x^2 + 1),
 (x + 1, x^3 + x^2 + x),
 (x^2 + x, x^4 + x^3 + 1),
 (x^3 + x^2, x^4 + x^2 + x),
 (x^4 + x^3, x^3 + x^2),
 (x^4 + x^2 + 1, x^4 + x^3 + x^2),
 (x^3 + x^2 + x + 1, x^4 + x^3),
 (x^4 + x^3 + x^2 + x, x^4 + x^2 + 1),
 (x^4 + x^3 + 1, x^3 + x),
 (x^4 + x^2 + x + 1, x^4 + x^2),
 (x^3 + x + 1, x^3 + 1),
 (x^4 + x^2 + x, x^4 + x),
 (x^3 + 1, 1),
 (x^4 + x, x + 1),
 (1, x^2 + x)]

In [125]:
def D3(a, b):
    return complex(ps2[toInt(a), toInt(b)]) * Z(a) @ X(b)

for p1 in points:
    for p2 in points:
        p_op = D3(p1[0], p1[1]) @ D3(p2[0], p2[1])
        s_op = D3(p1[0] + p2[0], p1[1] + p2[1])
        if not np.allclose(p_op, s_op):
            raise Exception(p1, p2)